# METACARPA Method Comparison

Comparing the current p-value transform method vs the Southam et al. 2017 beta-sign method
for estimating study correlation in meta-analysis with overlapping samples.

**Data:** GIANT 2022 Height EUR (with UKB) vs EUR (excluding UKB) — Yengo et al. 2022

**Bug report:** Brady Ryan (U Michigan) identified that the code uses `z_k = Phi^{-1}(1 - p_k)` for
dichotomization, ignoring effect direction, whereas the paper describes `z_k = Phi^{-1}(p_k/2) * sgn(beta_k)`.
The `--use-beta-sign` flag enables the paper's method. Allele harmonization in the first pass was also
added to ensure sign(beta) is correct when effect alleles differ between files.

In [ ]:
library(data.table)

current  <- fread("output_current.tsv")
betasign <- fread("output_betasign_fixed.tsv")
with_ukb <- fread("with_ukb_filtered.tsv")
no_ukb   <- fread("no_ukb_filtered.tsv")

# Force p-value columns to numeric (METACARPA outputs can be character for extreme values)
pcols <- c("p_wald", "p_corrected", "p_stouffer")
current[, (pcols) := lapply(.SD, as.numeric), .SDcols = pcols]
betasign[, (pcols) := lapply(.SD, as.numeric), .SDcols = pcols]

cat("Current method rows:", nrow(current), "\n")
cat("Beta-sign method rows:", nrow(betasign), "\n")
cat("With-UKB input rows:", nrow(with_ukb), "\n")
cat("No-UKB input rows:", nrow(no_ukb), "\n")

## 1. Allele harmonization check

The two GIANT files may use different effect alleles for the same variant.
This matters for Z-score sign and for the beta-sign method in METACARPA.

In [ ]:
# Merge input files on SNPID to check allele concordance
allele_check <- merge(
  with_ukb[, .(SNPID, EA1 = EFFECT_ALLELE, OA1 = OTHER_ALLELE, BETA1 = BETA)],
  no_ukb[, .(SNPID, EA2 = EFFECT_ALLELE, OA2 = OTHER_ALLELE, BETA2 = BETA)],
  by = "SNPID"
)

allele_check[, concordant := (EA1 == EA2)]
allele_check[, flipped := (EA1 == OA2 & OA1 == EA2)]

cat("Total overlapping variants:", nrow(allele_check), "\n")
cat("Concordant effect alleles:", sum(allele_check$concordant), 
    sprintf(" (%.1f%%)", 100 * mean(allele_check$concordant)), "\n")
cat("Flipped effect alleles:   ", sum(allele_check$flipped),
    sprintf(" (%.1f%%)", 100 * mean(allele_check$flipped)), "\n")
cat("Neither (ambiguous):      ", sum(!allele_check$concordant & !allele_check$flipped), "\n")

# Verify: when flipped, betas should have opposite signs
cat("\nAmong flipped variants, sign(beta1) == -sign(beta2):",
    mean(sign(allele_check[flipped == TRUE]$BETA1) == -sign(allele_check[flipped == TRUE]$BETA2)),
    "\n")

## 2. METACARPA correlation estimates

Two methods:
- **Current method** (p-value based): `b_transform(p) = Phi^{-1}(1-p) <= 0`. Unaffected by allele coding since p-values are invariant.
- **Beta-sign method** (Southam et al. 2017): `sign(beta) >= 0`. First pass harmonizes alleles between files.

In [ ]:
mat_current  <- readLines(list.files(pattern = "output_current.*matrix\\.txt$"))
mat_betasign <- readLines(list.files(pattern = "output_betasign_fixed.*matrix\\.txt$"))

rho_current  <- as.numeric(mat_current[3])
rho_betasign <- as.numeric(mat_betasign[3])

cat("METACARPA current (b_transform on p-value):", rho_current, "\n")
cat("METACARPA beta-sign (Southam et al. 2017): ", rho_betasign, "\n")

## 3. Empirical Z-score correlation (with allele harmonization)

Compute `cor(z1, z2)` after harmonizing effect alleles.
Stratify by p-value to isolate the null-SNP correlation
(which reflects pure sample overlap, free of genetic signal).

In [ ]:
# Build harmonized Z-scores
z_merged <- merge(
  with_ukb[, .(SNPID, EA1 = EFFECT_ALLELE, OA1 = OTHER_ALLELE,
               BETA1 = BETA, P1 = as.numeric(P), N1 = N)],
  no_ukb[, .(SNPID, EA2 = EFFECT_ALLELE, OA2 = OTHER_ALLELE,
              BETA2 = BETA, P2 = as.numeric(P), N2 = N)],
  by = "SNPID"
)

# Determine flip status and harmonize
z_merged[, flipped := (EA1 != EA2 & EA1 == OA2)]
z_merged[, BETA2_harm := ifelse(flipped, -BETA2, BETA2)]

# Signed Z-scores (using harmonized beta for sign)
z_merged[, z1 := sign(BETA1) * abs(qnorm(P1 / 2))]
z_merged[, z2 := sign(BETA2_harm) * abs(qnorm(P2 / 2))]

ok <- is.finite(z_merged$z1) & is.finite(z_merged$z2)

cat("=== Empirical cor(z1, z2) — HARMONIZED ===", "\n\n")

# Also compute the WRONG (unharmonized) correlation for comparison
z_merged[, z2_raw := sign(BETA2) * abs(qnorm(P2 / 2))]
rho_raw <- cor(z_merged$z1[ok], z_merged$z2_raw[ok])
rho_harm <- cor(z_merged$z1[ok], z_merged$z2[ok])
cat(sprintf("Unharmonized: cor = %.4f  (what the notebook showed before)\n", rho_raw))
cat(sprintf("Harmonized:   cor = %.4f\n\n", rho_harm))

cat("Stratified by significance (harmonized):\n")
thresholds <- c(1, 0.5, 0.1, 0.05, 0.01)
for (thr in thresholds) {
  idx <- ok & z_merged$P1 > thr & z_merged$P2 > thr
  r <- cor(z_merged$z1[idx], z_merged$z2[idx])
  cat(sprintf("  Both p > %-5s: cor = %.4f  (n = %d)\n", thr, r, sum(idx)))
}

# Best estimate of overlap-only correlation
null_idx <- ok & z_merged$P1 > 0.5 & z_merged$P2 > 0.5
rho_null <- cor(z_merged$z1[null_idx], z_merged$z2[null_idx])

## 4. Expected correlation from sample sizes

The formula $\rho = n_s / \sqrt{n_1 \cdot n_2}$ assumes single-cohort studies.
For meta-analyses of heterogeneous cohorts, this overestimates because
the effective contribution of shared samples depends on per-cohort weighting.
The empirical null-SNP correlation is a more reliable ground truth.

In [ ]:
n1_med <- median(z_merged$N1)
n2_med <- median(z_merged$N2)
rho_formula <- n2_med / sqrt(n1_med * n2_med)  # assumes n_shared = n2

# Back-calculate the effective n_shared from the empirical null correlation
n_shared_effective <- rho_null * sqrt(n1_med * n2_med)

cat("Median N (with UKB):", n1_med, "\n")
cat("Median N (excl UKB):", n2_med, "\n")
cat("\nNaive formula (n_shared = n2): rho =", round(rho_formula, 4),
    " <- overestimates for meta-analyses\n")
cat("Empirical null (both p > 0.5): rho =", round(rho_null, 4),
    " <- best ground truth\n")
cat("\nEffective shared N implied by empirical rho:",
    round(n_shared_effective), "\n")
cat("This is", round(100 * n_shared_effective / n2_med, 1),
    "% of the excl-UKB study N\n")

## 5. Correlation summary

In [ ]:
cat("==========================================\n")
cat("        CORRELATION COMPARISON            \n")
cat("==========================================\n")
cat(sprintf("Empirical null (p>0.5, harmonized): %.4f  <- ground truth\n", rho_null))
cat(sprintf("Empirical all SNPs (harmonized):    %.4f\n", rho_harm))
cat(sprintf("Empirical all SNPs (unharmonized):  %.4f  <- was wrong\n", rho_raw))
cat(sprintf("Naive formula (n_s/sqrt(n1*n2)):    %.4f  <- wrong for meta-analyses\n", rho_formula))
cat(sprintf("METACARPA current (b_transform):    %.4f\n", rho_current))
cat(sprintf("METACARPA beta-sign (Southam 2017): %.4f\n", rho_betasign))
cat("==========================================\n")

## 6. Lambda GC

In [ ]:
lambda_gc <- function(p) {
  p <- as.numeric(p)
  p <- p[!is.na(p) & p > 0 & p < 1]
  chi2 <- qnorm(p / 2)^2
  median(chi2) / qchisq(0.5, 1)
}

results <- data.table(
  p_column = c("p_wald", "p_corrected", "p_stouffer"),
  lambda_current = c(
    lambda_gc(current$p_wald),
    lambda_gc(current$p_corrected),
    lambda_gc(current$p_stouffer)
  ),
  lambda_betasign = c(
    lambda_gc(betasign$p_wald),
    lambda_gc(betasign$p_corrected),
    lambda_gc(betasign$p_stouffer)
  )
)

results[, diff := lambda_betasign - lambda_current]
print(results)

cat("\nNote: lambda >> 1 for all methods because height is extremely polygenic.\n")
cat("The current method estimates rho lower, so corrects less aggressively.\n")
cat("Null simulation (section 8) shows the current method STILL undercorrects.\n")

## 7. QQ plots

In [ ]:
qqplot_p <- function(p, main = "", col = "black", add = FALSE, ...) {
  p <- as.numeric(p)
  p <- sort(p[!is.na(p) & p > 0 & p < 1])
  n <- length(p)
  expected <- -log10((1:n) / (n + 1))
  observed <- -log10(p)
  if (!add) {
    plot(expected, observed, pch = 20, cex = 0.3, col = col,
         xlab = expression(Expected ~ -log[10](p)),
         ylab = expression(Observed ~ -log[10](p)),
         main = main, ...)
    abline(0, 1, col = "grey40", lty = 2)
  } else {
    points(expected, observed, pch = 20, cex = 0.3, col = col)
  }
}

par(mfrow = c(1, 3), mar = c(4, 4, 3, 1))

qqplot_p(current$p_wald, main = "p_wald", col = "#D55E00")
qqplot_p(betasign$p_wald, col = "#0072B2", add = TRUE)
qqplot_p(current$p_stouffer, col = "grey60", add = TRUE)
legend("topleft", legend = c(
  paste0("Current (rho=", round(rho_current, 3), ")"),
  paste0("Beta-sign (rho=", round(rho_betasign, 3), ")"),
  "Uncorrected"),
  col = c("#D55E00", "#0072B2", "grey60"), pch = 20, cex = 0.8, bty = "n")

qqplot_p(current$p_corrected, main = "p_corrected", col = "#D55E00")
qqplot_p(betasign$p_corrected, col = "#0072B2", add = TRUE)
qqplot_p(current$p_stouffer, col = "grey60", add = TRUE)
legend("topleft", legend = c("Current", "Beta-sign", "Uncorrected"),
  col = c("#D55E00", "#0072B2", "grey60"), pch = 20, cex = 0.8, bty = "n")

qqplot_p(current$p_wald, main = "p_wald (zoomed)",
         col = "#D55E00", xlim = c(0, 3), ylim = c(0, 5))
qqplot_p(betasign$p_wald, col = "#0072B2", add = TRUE)
qqplot_p(current$p_stouffer, col = "grey60", add = TRUE)
legend("topleft", legend = c("Current", "Beta-sign", "Uncorrected"),
  col = c("#D55E00", "#0072B2", "grey60"), pch = 20, cex = 0.8, bty = "n")

## 8. Null simulation

The real data is contaminated by signal (height is extremely polygenic), inflating both methods.
Simulate correlated null summary statistics with known overlap $\rho$,
then check which method recovers $\rho$ and controls type 1 error.

Approach: draw $(z_1, z_2) \sim BVN(0, 0, 1, 1, \rho)$, convert to
p-values and betas, write METACARPA-format files, run both methods.

**Pre-computed results** (100k null variants, rho = 0.0 to 0.7):

| True rho | Current rho-hat | Beta-sign rho-hat | Current lambda | Beta-sign lambda | Uncorrected lambda |
|----------|-----------------|-------------------|---------------|------------------|--------------------|
| 0.0 | -0.005 | 0.006 | 1.017 | 1.011 | 1.014 |
| 0.1 | **0.008** | 0.097 | 1.090 | 1.042 | 1.095 |
| 0.2 | **0.024** | 0.189 | 1.178 | 1.089 | 1.195 |
| 0.3 | **0.054** | 0.284 | 1.258 | 1.129 | 1.294 |
| 0.5 | **0.159** | 0.475 | 1.375 | 1.181 | 1.493 |
| 0.7 | **0.360** | 0.670 | 1.401 | 1.185 | 1.692 |

**Conclusion:** The current method severely underestimates rho, leading to undercorrection
and inflated type 1 error. The beta-sign method recovers rho much more accurately.

In [ ]:
simulate_gwas <- function(n_variants, rho, n1 = 10000, n2 = 10000, seed = 42) {
  set.seed(seed)
  # Correlated Z-scores under the null
  Sigma <- matrix(c(1, rho, rho, 1), 2, 2)
  z <- MASS::mvrnorm(n_variants, mu = c(0, 0), Sigma = Sigma)
  
  # Convert to GWAS summary statistics
  se1 <- rep(1 / sqrt(n1), n_variants)
  se2 <- rep(1 / sqrt(n2), n_variants)
  beta1 <- z[, 1] * se1
  beta2 <- z[, 2] * se2
  p1 <- 2 * pnorm(-abs(z[, 1]))
  p2 <- 2 * pnorm(-abs(z[, 2]))
  
  # Build METACARPA-format data.tables
  chrs <- rep(1:22, length.out = n_variants)
  pos  <- ave(seq_len(n_variants), chrs, FUN = function(x) seq_along(x) * 1000)
  
  common <- data.table(
    SNPID = paste0(chrs, ":", pos, ":A:G"),
    RSID  = paste0("rs", seq_len(n_variants)),
    CHR   = chrs,
    POS   = as.integer(pos)
  )
  
  study1 <- cbind(common, data.table(
    EFFECT_ALLELE = "A", OTHER_ALLELE = "G",
    EFFECT_ALLELE_FREQ = 0.3,
    BETA = beta1, SE = se1, P = p1, N = n1
  ))
  
  study2 <- cbind(common, data.table(
    EFFECT_ALLELE = "A", OTHER_ALLELE = "G",
    EFFECT_ALLELE_FREQ = 0.3,
    BETA = beta2, SE = se2, P = p2, N = n2
  ))
  
  # Sort by chr, pos
  setorder(study1, CHR, POS)
  setorder(study2, CHR, POS)
  
  list(study1 = study1, study2 = study2, z = z, rho_true = rho)
}

cat("simulate_gwas() ready.\n")
cat("Usage: sim <- simulate_gwas(n_variants = 100000, rho = 0.3)\n")

In [ ]:
run_metacarpa <- function(sim, method = "current", metacarpa_bin = "../src/metacarpa",
                          lib_path = "../src/lib") {
  tmpdir <- tempdir()
  f1 <- file.path(tmpdir, "sim_study1.tsv")
  f2 <- file.path(tmpdir, "sim_study2.tsv")
  fout <- file.path(tmpdir, paste0("sim_output_", method, ".tsv"))
  
  fwrite(sim$study1, f1, sep = "\t")
  fwrite(sim$study2, f2, sep = "\t")
  
  flag <- if (method == "betasign") "--use-beta-sign" else ""
  
  cmd <- sprintf(
    "LD_LIBRARY_PATH=%s %s -I %s -I %s -O %s -t '\t' -c 3 -q 4 -u 5 -v 6 -a 7 -b 8 -s 9 -p 10 -n 11 -i 2 %s 2>&1",
    lib_path, metacarpa_bin, f1, f2, fout, flag
  )
  
  output <- system(cmd, intern = TRUE)
  
  # Find and read matrix file
  mat_files <- list.files(tmpdir, pattern = paste0("sim_output_", method, ".*matrix\\.txt$"),
                          full.names = TRUE)
  rho_est <- NA
  if (length(mat_files) > 0) {
    mat_lines <- readLines(mat_files[1])
    rho_est <- as.numeric(mat_lines[3])
  }
  
  # Read output
  result <- NULL
  if (file.exists(fout)) {
    result <- fread(fout)
    pcols <- c("p_wald", "p_corrected", "p_stouffer")
    result[, (pcols) := lapply(.SD, as.numeric), .SDcols = pcols]
  }
  
  # Cleanup matrix files
  file.remove(mat_files)
  
  list(rho_est = rho_est, result = result, log = output)
}

cat("run_metacarpa() ready.\n")

In [ ]:
# Simulate and run for several rho values
rho_values <- c(0.0, 0.1, 0.2, 0.3, 0.5, 0.7)
n_variants <- 100000

sim_results <- data.table(
  rho_true = numeric(),
  rho_current = numeric(),
  rho_betasign = numeric(),
  lambda_wald_current = numeric(),
  lambda_wald_betasign = numeric(),
  lambda_stouffer = numeric(),
  t1e_wald_current = numeric(),
  t1e_wald_betasign = numeric(),
  t1e_stouffer = numeric()
)

for (rho in rho_values) {
  cat(sprintf("\n=== rho = %.1f ===", rho), "\n")
  sim <- simulate_gwas(n_variants, rho, seed = 123)
  
  res_cur <- run_metacarpa(sim, "current")
  res_bs  <- run_metacarpa(sim, "betasign")
  
  cat(sprintf("  Estimated rho: current = %.4f, beta-sign = %.4f\n",
              res_cur$rho_est, res_bs$rho_est))
  
  # Lambda GC and type 1 error at alpha = 0.05
  lam_cur <- lambda_gc(res_cur$result$p_wald)
  lam_bs  <- lambda_gc(res_bs$result$p_wald)
  lam_unc <- lambda_gc(res_cur$result$p_stouffer)
  
  t1e_cur <- mean(res_cur$result$p_wald < 0.05, na.rm = TRUE)
  t1e_bs  <- mean(res_bs$result$p_wald < 0.05, na.rm = TRUE)
  t1e_unc <- mean(as.numeric(res_cur$result$p_stouffer) < 0.05, na.rm = TRUE)
  
  cat(sprintf("  Lambda GC (p_wald): current = %.3f, beta-sign = %.3f, uncorrected = %.3f\n",
              lam_cur, lam_bs, lam_unc))
  cat(sprintf("  Type 1 error (p_wald < 0.05): current = %.4f, beta-sign = %.4f, uncorrected = %.4f\n",
              t1e_cur, t1e_bs, t1e_unc))
  
  sim_results <- rbind(sim_results, data.table(
    rho_true = rho,
    rho_current = res_cur$rho_est,
    rho_betasign = res_bs$rho_est,
    lambda_wald_current = lam_cur,
    lambda_wald_betasign = lam_bs,
    lambda_stouffer = lam_unc,
    t1e_wald_current = t1e_cur,
    t1e_wald_betasign = t1e_bs,
    t1e_stouffer = t1e_unc
  ))
}

cat("\n")
print(sim_results)

In [ ]:
# Plot results
par(mfrow = c(1, 3), mar = c(4, 4, 3, 1))

# 1. Estimated vs true rho
plot(sim_results$rho_true, sim_results$rho_current, type = "b",
     pch = 19, col = "#D55E00", lwd = 2,
     xlab = expression("True " * rho), ylab = expression("Estimated " * rho),
     main = "Correlation recovery",
     xlim = c(0, 0.8), ylim = c(0, 0.8))
lines(sim_results$rho_true, sim_results$rho_betasign, type = "b",
      pch = 17, col = "#0072B2", lwd = 2)
abline(0, 1, col = "grey40", lty = 2)
legend("topleft", legend = c("Current", "Beta-sign"),
       col = c("#D55E00", "#0072B2"), pch = c(19, 17), lwd = 2, bty = "n")

# 2. Lambda GC
yr <- range(c(sim_results$lambda_wald_current, sim_results$lambda_wald_betasign,
              sim_results$lambda_stouffer), na.rm = TRUE)
plot(sim_results$rho_true, sim_results$lambda_wald_current, type = "b",
     pch = 19, col = "#D55E00", lwd = 2,
     xlab = expression("True " * rho), ylab = expression("Lambda"[GC]),
     main = "Genomic inflation",
     xlim = c(0, 0.8), ylim = yr)
lines(sim_results$rho_true, sim_results$lambda_wald_betasign, type = "b",
      pch = 17, col = "#0072B2", lwd = 2)
lines(sim_results$rho_true, sim_results$lambda_stouffer, type = "b",
      pch = 15, col = "grey60", lwd = 2)
abline(h = 1, col = "grey40", lty = 2)
legend("topleft", legend = c("Current", "Beta-sign", "Uncorrected"),
       col = c("#D55E00", "#0072B2", "grey60"), pch = c(19, 17, 15), lwd = 2, bty = "n")

# 3. Type 1 error at alpha = 0.05
yr <- range(c(sim_results$t1e_wald_current, sim_results$t1e_wald_betasign,
              sim_results$t1e_stouffer), na.rm = TRUE)
plot(sim_results$rho_true, sim_results$t1e_wald_current, type = "b",
     pch = 19, col = "#D55E00", lwd = 2,
     xlab = expression("True " * rho), ylab = "Type 1 error rate",
     main = expression("Type 1 error (" * alpha * " = 0.05)"),
     xlim = c(0, 0.8), ylim = yr)
lines(sim_results$rho_true, sim_results$t1e_wald_betasign, type = "b",
      pch = 17, col = "#0072B2", lwd = 2)
lines(sim_results$rho_true, sim_results$t1e_stouffer, type = "b",
      pch = 15, col = "grey60", lwd = 2)
abline(h = 0.05, col = "grey40", lty = 2)
legend("topleft", legend = c("Current", "Beta-sign", "Uncorrected"),
       col = c("#D55E00", "#0072B2", "grey60"), pch = c(19, 17, 15), lwd = 2, bty = "n")